In [ ]:
!pip install  -q langchain langgraph transformers bitsandbytes langchain-huggingface langchain-community chromadb

In [ ]:
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

docs_text = """
火影代數	姓名	師傅	徒弟
初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
"""

docs = [Document(page_content=txt.strip()) for txt in docs_text.strip().split("\n\n")]

# chromadb 預設使用的大型語言模型為 "all-MiniLM-L6-v2"，由於該大型語言模型不支持中文，所以將模型替換為 "infgrad/stella-base-zh-v3-1792d"，並對 embedding 進行量化
embedding_model = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    encode_kwargs={"normalize_embeddings": True}
)

persist_path = "document_store"
collection_name = "naruto_collection"
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=persist_path,
    collection_name=collection_name
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

# 使用 4-bit 量化模型
model_id = "MediaTek-Research/Breeze-7B-Instruct-v1_0"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
)

# 載入 tokenizer 與 4-bit 模型
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

### 1. 模型載入

使用的模型與 chatbot 教學同一顆

In [ ]:
generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.4,
    return_full_text=False # 僅返回生成的回應內容
)

Device set to use cuda:0


### 2. 定義狀態（State）

這裡定義的 RAGState，包含了查詢文字（query）、檢索到的文件（docs）、以及最後生成的回答（answer）。

In [ ]:
from typing_extensions import TypedDict, List

# 定義 LangGraph 的 State 結構
class RAGState(TypedDict):
    query: str
    docs: List[Document]
    answer: str

### 3. 定義節點（Node）

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def retrieve_node(state: RAGState) -> RAGState:
    query = state["query"]
    # similarity_search 距離越小越相似
    docs = vectorstore.similarity_search(query, k=3)
    return {"query": query, "docs": docs, "answer": ""}

def generate_node(state: RAGState) -> RAGState:
    query, docs = state["query"], state["docs"]
    context = "\n".join([d.page_content for d in docs])
    prompt = (
        f"你是一個知識型助手，請根據以下內容回答問題：\n\n"
        f"內容：{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": docs, "answer": output}

def direct_generate_node(state: RAGState) -> RAGState:
    query = state["query"]
    prompt = f"請回答以下問題：{query}\n\n回答："
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": [], "answer": output}

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-mpnet-base-v2')
# 定義 Route Node（決定走哪條路）
def route_by_query(state):



    query = state["query"]
    # 將字句轉換為向量
    emb1 = model.encode(docs_text, convert_to_tensor=True)
    emb2 = model.encode(query, convert_to_tensor=True)
    # 計算相似度
    path_from_cosine = util.pytorch_cos_sim(emb1, emb2).item()
    print(f'cosine_similarity {path_from_cosine}')
    choice = "naruto" if path_from_cosine > 0.72 else "general"
    print(f"跑到 → {choice}")
    return choice

### 4. 建立 LangGraph 流程圖（StateGraph）

在 LangGraph / LangChain 裡，RunnableLambda 就是用來把一個普通的 Python 函式（function）包裝成一個「Runnable」物件。

Runnable 是 LangChain 裡的一個「標準介面」（Protocol Interface），代表「這個東西可以 .invoke()、可以被執行」。

> ※ 換句話說：RunnableLambda 讓普通函式能接到 LangGraph 的 Node 上運行。

為什麼第一個例子 chatbot 不需要使用 RunnableLambda ?

- 如果只是單線到底（直直連接），不管有幾個節點，都不用自己包 RunnableLambda。

- 只要涉及「判斷分流」、「根據條件走不同路線」，就一定要自己把判斷的 function 包成 RunnableLambda

In [ ]:
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

# 建立 LangGraph 流程圖
graph_builder = StateGraph(RAGState)

graph_builder.set_entry_point("condition")
graph_builder.add_node("condition", RunnableLambda(lambda x: x))  # 進來就分流，不改變內容
graph_builder.add_node("retriever", RunnableLambda(retrieve_node))
graph_builder.add_node("generator", RunnableLambda(generate_node))
graph_builder.add_node("direct_generator", RunnableLambda(direct_generate_node))

# 設定條件分流
graph_builder.add_conditional_edges(
    source="condition",
    path=RunnableLambda(route_by_query),
    path_map={
        "naruto": "retriever",
        "general": "direct_generator",
    }
)

# 接下來的正常連接
graph_builder.add_edge("retriever", "generator")
graph_builder.add_edge("generator", END)
graph_builder.add_edge("direct_generator", END)

# 編譯 Graph
graph = graph_builder.compile()

### 6. RAG 結果

In [ ]:
print("開始對話吧（輸入 q 結束）")

while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    # 設定初始 State
    init_state: RAGState = {
        "query": user_input,
        "docs": [],
        "answer": ""
    }
    # 呼叫 LangGraph
    result = graph.invoke(init_state)
    raw_output = result["answer"]

    answer_text = raw_output.split("回答：")[-1].strip()
    print("回答：", answer_text)
    print("===" * 20, "\n")

開始對話吧（輸入 q 結束）
使用者: 誰是第四代火影?
cosine_similarity 0.7422103881835938
跑到 → naruto
回答： 第四代火影是波風湊。

使用者: 第四代火影的師傅是誰?
cosine_similarity 0.7497969269752502
跑到 → naruto
回答： 第四代火影的師傅是自來也。

使用者: 第四代火影的徒弟有哪些人?


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


cosine_similarity 0.752856433391571
跑到 → naruto
回答： 第四代火影的徒弟有旗木卡卡西、宇智波帶土、野原琳。

使用者: 相對論是誰發明的?
cosine_similarity 0.5832839012145996
跑到 → general
回答： 相對論是愛因斯坦（Albert Einstein）在 1905 年提出的。

使用者: q
掰啦！


# advance

改成能支援多輪問答（Multi-turn RAG），並能根據前面的query判斷問題。

> 請將 RAGState 加入 history 欄位，並在生成回答時，將歷史對話與當前問題一起組成 prompt。

> Hint：
```
class MultiTurnRAGState(TypedDict):  
    history: List[str]  
    query: str  
    docs: List[Document]  
    answer: str
```



In [ ]:
!pip install -q langchain langgraph transformers bitsandbytes langchain-huggingface langchain-community chromadb

In [ ]:
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

docs_text = """
火影代數	姓名	師傅	徒弟
初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
"""

docs = [Document(page_content=txt.strip()) for txt in docs_text.strip().split("\n\n")]

# chromadb 預設使用的大型語言模型為 "all-MiniLM-L6-v2"，由於該大型語言模型不支持中文，所以將模型替換為 "infgrad/stella-base-zh-v3-1792d"，並對 embedding 進行量化
embedding_model = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    encode_kwargs={"normalize_embeddings": True}
)

persist_path = "document_store"
collection_name = "naruto_collection"
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=persist_path,
    collection_name=collection_name
    #,collection_metadata={"hnsw:space": "cosine"}
)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

# 使用 4-bit 量化模型
model_id = "MediaTek-Research/Breeze-7B-Instruct-v1_0"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
)

# 載入 tokenizer 與 4-bit 模型
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)
generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.4,
    return_full_text=False # 僅返回生成的回應內容
)


In [8]:
from typing_extensions import TypedDict, List
# 定義 LangGraph 的 State 結構
class MultiTurnRAGState(TypedDict):
    history: List[str]
    query: str
    docs: List[Document]
    answer: str
from sklearn.metrics.pairwise import cosine_similarity
def retrieve_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query = state["query"]
    docs = vectorstore.similarity_search(query, k=3)
    print({
        "query": query,
        "docs": docs,
        "answer": "",
        "history": state["history"]
    })
    return {
        "query": query,
        "docs": docs,
        "answer": "",
        "history": state["history"]
    }

def generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query, docs, history = state["query"], state["docs"], state["history"]
    context = "\n".join([d.page_content for d in docs])

    history_prompt = "\n".join([f"Q: {history[i]}\nA: {history[i+1]}" for i in range(0, len(history), 2)])
    full_prompt = (
        f"你是一個知識型助手，請根據以下歷史對話和內容回答問題：\n\n"
        f"歷史對話：\n{history_prompt}\n\n"
        f"內容：\n{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = generator(full_prompt, max_new_tokens=200)[0]["generated_text"]
    return {
        "query": query,
        "docs": docs,
        "answer": output,
        "history": history + [query, output]
    }

def direct_generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query, history = state["query"], state["history"]

    history_prompt = "\n".join([f"Q: {history[i]}\nA: {history[i+1]}" for i in range(0, len(history), 2)])
    prompt = (
        f"你是一個聊天助手，請根據以下歷史對話回答問題：\n\n"
        f"{history_prompt}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {
        "query": query,
        "docs": [],
        "answer": output,
        "history": history + [query, output]
    }

from sentence_transformers import SentenceTransformer, util

ebd_consin_model = SentenceTransformer('paraphrase-mpnet-base-v2')
# 定義 Route Node（決定走哪條路）
def route_by_query(state):



    query = state["query"]
    # 將字句轉換為向量
    emb1 = ebd_consin_model.encode(docs_text, convert_to_tensor=True)
    emb2 = ebd_consin_model.encode(query, convert_to_tensor=True)
    # 計算相似度
    path_from_cosine = util.pytorch_cos_sim(emb1, emb2).item()
    print(f'cosine_similarity {path_from_cosine}')
    choice = "naruto" if path_from_cosine > 0.5 else "general"
    print(f"跑到 → {choice}")
    return choice
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

# 建立 LangGraph 流程圖
graph_builder = StateGraph(MultiTurnRAGState)

graph_builder.set_entry_point("condition")
graph_builder.add_node("condition", RunnableLambda(lambda x: x))  # 進來就分流，不改變內容
graph_builder.add_node("retriever", RunnableLambda(retrieve_node))
graph_builder.add_node("generator", RunnableLambda(generate_node))
graph_builder.add_node("direct_generator", RunnableLambda(direct_generate_node))

# 設定條件分流
graph_builder.add_conditional_edges(
    source="condition",
    path=RunnableLambda(route_by_query),
    path_map={
        "naruto": "retriever",
        "general": "direct_generator",
    }
)

# 接下來的正常連接
graph_builder.add_edge("retriever", "generator")
graph_builder.add_edge("generator", END)
graph_builder.add_edge("direct_generator", END)

# 編譯 Graph
graph = graph_builder.compile()

In [ ]:
global_history: List[str] = []

print("開始對話吧（輸入 q 結束）")
while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    state = {"history": global_history, "query": user_input, "docs": [], "answer": ""}
    result = graph.invoke(state)

    answer = result["answer"].split("回答：")[-1].strip()
    print("AI 助理:", answer)
    print("=" * 120, "\n")

    global_history = result["history"]


開始對話吧（輸入 q 結束）
使用者: 第四代火影是誰?
cosine_similarity 0.7470983266830444
跑到 → naruto
{'query': '第四代火影是誰?', 'docs': [Document(metadata={}, page_content='火影代數\t姓名\t師傅\t徒弟\n初代\t千手柱間\t無明確記載\t猿飛日斬、水戶門炎、轉寢小春\n二代\t千手扉間\t千手柱間（兄長）\t猿飛日斬、志村團藏、宇智波鏡等\n三代\t猿飛日斬\t千手柱間、千手扉間\t自來也、大蛇丸、千手綱手（傳說三忍）\n四代\t波風湊\t自來也\t旗木卡卡西、宇智波帶土、野原琳\n五代\t千手綱手\t猿飛日斬\t春野櫻、志乃等（主要為春野櫻）\n六代\t旗木卡卡西\t波風湊\t漩渦鳴人、宇智波佐助、春野櫻（第七班）\n七代\t漩渦鳴人\t自來也、旗木卡卡西\t木葉丸等（主要為木葉丸）'), Document(metadata={}, page_content='火影代數\t姓名\t師傅\t徒弟\n初代\t千手柱間\t無明確記載\t猿飛日斬、水戶門炎、轉寢小春\n二代\t千手扉間\t千手柱間（兄長）\t猿飛日斬、志村團藏、宇智波鏡等\n三代\t猿飛日斬\t千手柱間、千手扉間\t自來也、大蛇丸、千手綱手（傳說三忍）\n四代\t波風湊\t自來也\t旗木卡卡西、宇智波帶土、野原琳\n五代\t千手綱手\t猿飛日斬\t春野櫻、志乃等（主要為春野櫻）\n六代\t旗木卡卡西\t波風湊\t漩渦鳴人、宇智波佐助、春野櫻（第七班）\n七代\t漩渦鳴人\t自來也、旗木卡卡西\t木葉丸等（主要為木葉丸）')], 'answer': '', 'history': []}
AI 助理: 第四代火影是波風湊。

使用者: 他的師父是誰?
cosine_similarity 0.6340187191963196
跑到 → naruto
{'query': '他的師父是誰?', 'docs': [Document(metadata={}, page_content='火影代數\t姓名\t師傅\t徒弟\n初代\t千手柱間\t無明確記載\t猿飛日斬、水戶門炎、轉寢小春\n二代\t千手扉間\t千手柱間（兄長）\t猿飛日斬、志村團藏、宇智波鏡等\n三代\t猿飛日斬\t千手柱間、千手扉間\